In [ ]:
from elasticsearch import helpers
from elasticsearch import Elasticsearch
from datetime import datetime
import json
from elasticsearch_reader import ElasticsearchReader as esr
from elasticsearch_creator import ElasticsearchCreator as esc

In [ ]:
with open("test_data.json","r") as file:
    _actions = json.loads(file.read())
result_list = esc.bulk_load(_actions)
bulk_result = esc.process_result(result_list)
bulk_result

In [ ]:
esr.get_last_id('inventory',"document")

In [ ]:
%%bash
curl -X GET "localhost:9200/_search?pretty" -H 'Content-Type: application/json' -d'
{
    "query": {
        "function_score": {
            "query": {
                "match": { "message": "elasticsearch" }
            },
            "script_score" : {
                "script" : {
                  "source": "Math.log(2 + doc[\u0027likes\u0027].value)"
                }
            }
        }
    }
}'

In [ ]:
%%bash
curl -X GET "localhost:9200/_search?pretty" -H 'Content-Type: application/json' -d'
{
    "query": {
        "match" : {
            "message" : "elasticsearch"
        }
    }
}'

In [ ]:
%%bash
curl -X GET "localhost:9200/_search?pretty" -H 'Content-Type: application/json' -d'
{
    "query": {
        "match_phrase" : {
            "message" : "elasticsearch"
        }
    }
}'

In [ ]:
%%bash
curl -XDELETE 'localhost:9200/my_index?pretty'
curl -X PUT "localhost:9200/my_index" -H 'Content-Type: application/json' -d'
{
  "mappings": {
    "_doc": {
      "properties": {
        "location": {
          "type": "geo_point"
        }
      }
    }
  }
}'

In [ ]:
%%bash
curl -X PUT "localhost:9200/my_index/_doc/1?pretty" -H 'Content-Type: application/json' -d'
{
  "text": "Geo-point as an object",
  "location": { 
    "lat":19.438664,
    "lon": -99.183666
  }
}'
curl -X PUT "localhost:9200/my_index/_doc/2?pretty" -H 'Content-Type: application/json' -d'
{
  "text": "Geo-point as an object",
  "location": { 
    "lat": 19.444501,
    "lon": -99.183022
  }
}'
curl -X PUT "localhost:9200/my_index/_doc/3?pretty" -H 'Content-Type: application/json' -d'
{
  "text": "Geo-point as an object",
  "location": { 
    "lat": 19.439716,
    "lon": -99.183752
  }
}'

In [ ]:
%%bash
#-99.1879761219,19.4359329685,-99.1779339314,19.4419628533
curl -X GET "localhost:9200/_search?pretty" -H 'Content-Type: application/json' -d'
{
    "query": {
        "bool" : {
            "must" : {
                "match_all" : {}
            },
            "filter" : {
                "geo_bounding_box" : {
                    "location" : {
                        "top_left" : {
                            "lat" : 19.4419628533,
                            "lon" : -99.1879761219
                        },
                        "bottom_right" : {
                            "lat" : 19.4359329685,
                            "lon" : -99.1779339314
                        }
                    }
                }
            }
        }
    }
}'